# Predicting Population

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

In [ ]:
# Dataset class
class SatelliteDataset(Dataset):
  def __init__(self, size=256, train=True):
    self.train = train
    self.size = size
    self.root = os.getcwd() + "/drive/MyDrive/Deep Learning Project"

  def __getitem__(self,index):
    
    return img[0],label[0][0]
  
  def __len__(self):
    return len(self.dataset_folder)


In [ ]:
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, up_sample=False):
    super(ConvBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, (3,3), padding=(1,1))
    self.conv2 = nn.Conv2d(out_channels, out_channels, (3,3), padding=(1,1))
    self.up_sample = up_sample
    if up_sample:    
      # Up sample with up-conv 2x2, doubling along each dimension
      self.up = nn.ConvTranspose2d(out_channels, out_channels//2, 2, stride=2)
    # We do not want to down sample in the block, since we need the non-down-sampled output for skip connections

  def forward(self, input):
    x = F.relu(self.conv1(input))
    x = F.relu(self.conv2(x))
    if self.up_sample:
      x = self.up(x)
    return x

class SatelliteDetection(nn.Module):
  def __init__(self):
    super(SatelliteDetection, self).__init__()
    # Downsample convolution blocks
    self.dblock1 = ConvBlock(3, 64)
    self.dblock2 = ConvBlock(64, 128)
    self.dblock3 = ConvBlock(128, 256)
    self.dblock4 = ConvBlock(256, 512)
    # Upsample convolution blocks
    self.ublock1 = ConvBlock(512, 1024, True)
    # The next 4 blocks have doubled input channels due to concatination of skip connections
    self.ublock2 = ConvBlock(1024, 512, True)
    self.ublock3 = ConvBlock(512, 256, True)
    self.ublock4 = ConvBlock(256, 128, True)
    # Output "block"
    self.convf1 = nn.Conv2d(128, 64, (3,3), padding=(1,1))
    self.convf2 = nn.Conv2d(64, 64, (3,3), padding=(1,1))
    self.convf3 = nn.Conv2d(64, 2, (1,1), padding=(0,0))

    self.down = nn.MaxPool2d(2)
 
  def forward(self, input):
    # Save the last feature maps on each level! Pass an instance forward, but keep the variable referencing the same feature map for concatenation
    l1 = self.dblock1(input)
    l2 = self.dblock2(self.down(l1))
    l3 = self.dblock3(self.down(l2))
    l4 = self.dblock4(self.down(l3))
    # Concatinate l1 - l4 on inputs across the U in reverse order, matching sizes
    u = self.ublock1(self.down(l4))
    u = self.ublock2(torch.cat((l4, u), dim=1))
    u = self.ublock3(torch.cat((l3, u), dim=1))
    u = self.ublock4(torch.cat((l2, u), dim=1))
    out = F.relu(self.convf1(torch.cat((l1, u), dim=1)))
    out = F.relu(self.convf2(out))
    out = self.convf3(out)
    return out

In [ ]:
# Initialize Datasets
train_dataset = SatelliteDataset(train=True, size=96)
val_dataset = SatelliteDataset(train=False, size=96)

# Grab test image and label
test_im, test_gt = val_dataset[172]
# Give test image a batch dim of 1 and put on GPU
test_im = test_im.unsqueeze(0).cuda()

# Initialize Model
model = SatelliteDetection()
model.cuda()

# Initialize Objective and Optimizer and other parameters
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_losses = []
val_losses = []
train_accs = []
val_accs = []
train_accs_pw = []
val_accs_pw = []
test_preds = []

import time

def scope():
  try:
    #your code for calling dataset and dataloader
    gc.collect()
    print(torch.cuda.memory_allocated(0) / 1e9)

    # Initialize DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=6, pin_memory=True, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=6, pin_memory=True, shuffle=True)

    # Call your model, figure out loss and accuracy

    # get the mean of losses from running the validation loader
    vals = []
    val_a = []
    val_a_pw = 0
    with torch.no_grad():
      for x, y_truth in val_loader:
        gc.collect()
        x, y_truth = x.cuda(), y_truth.cuda() 
        y_hat = model(x.cuda())
        vals.append(objective(y_hat, y_truth.long()).item())
        iou = IoU(torch.softmax(y_hat,1), y_truth.float())
        val_a_pw += pixelwise_accuracy(y_hat, y_truth).sum().item()
        if iou != -1:
          val_a.append(iou)
      val_losses.append((len(train_losses), np.mean(vals)))
      val_accs.append((len(train_losses), np.mean(val_a)))
      val_accs_pw.append(val_a_pw / len(val_dataset))

    num_epochs = 5
    for epoch in range(num_epochs):
      loop = tqdm(total=len(train_loader), position=0, leave=False)

      for batch, (x, y_truth) in enumerate(train_loader):
        gc.collect()
        x, y_truth = x.cuda(), y_truth.cuda()

        optimizer.zero_grad()
        y_hat = model(x)

        loss = objective(y_hat, y_truth.long())
        
        loss.backward()

        train_losses.append(loss.item())
        train_acc = IoU(torch.softmax(y_hat, 1), y_truth.float())
        if train_acc != -1:
          train_accs.append(train_acc)
          train_accs_pw.append(pixelwise_accuracy(y_hat, y_truth).mean())
        loop.set_description('epoch:{0}, loss{1:.4f}, accuracy:{2:.3f}'.format(epoch, loss.item(), train_acc))
        loop.update(1)

        optimizer.step()

      # get the mean of losses from running the validation loader
      vals = []
      val_a = []
      val_a_pw = 0
      with torch.no_grad():
        for x, y_truth in val_loader:
          gc.collect()
          x, y_truth = x.cuda(), y_truth.cuda() 
          y_hat = model(x.cuda())
          vals.append(objective(y_hat, y_truth.long()).item())
          iou = IoU(torch.softmax(y_hat,1), y_truth.float())
          val_a_pw += pixelwise_accuracy(y_hat, y_truth).sum().item()
          if iou != -1:
            val_a.append(iou)
        val_losses.append((len(train_losses), np.mean(vals)))
        val_accs.append((len(train_losses), np.mean(val_a)))
        val_accs_pw.append(val_a_pw / len(val_dataset))

        # Collect predictions on the test image
        test_preds.append(model(test_im).cpu())

      loop.close()
      
    
  except:
    __ITB__()
    
begin = time.time()
scope()
print("Runtime =", time.time()-begin)